<a href="https://colab.research.google.com/github/v1an1/Remote-sensing-image-classification/blob/master/RS_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount your drive containing 'src.zip'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Unzip the folder

In [0]:
!unzip '/content/drive/My Drive/src.zip'

Copy the model file in current directory for usage

In [0]:
!cp "/content/src/utils_jnb.py" .

In [2]:
import torch 
import matplotlib.pylab as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchsummary import summary
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import os
import time
from utils_jnb import *

torch.manual_seed(0)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
directory = "/content/src"

In [0]:
num_classes = 3

vgg19 = models.vgg19_bn(pretrained=True)
# Freeze model parameters
# Same for all
for param in vgg19.parameters():
    param.requires_grad = False

vgg19.classifier = nn.Sequential(nn.Linear(25088, 4096),
nn.ReLU(),
nn.Dropout(0.4),
nn.Linear(4096, 1024),
nn.ReLU(),
nn.Dropout(0.4),
nn.Linear(1024, num_classes),
nn.LogSoftmax(dim=1))

In [0]:
# Define Optimizer and Loss Function
lossFunc = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg19.parameters(), lr=1e-2)

# Decay LR by a factor of 0.1 every 7 epochs
expLrScheduler = lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)

vgg19.to(device)

In [0]:
summary(vgg19, input_size=(3, 224, 224), batch_size=32)

In [0]:
history = trainValid(vgg19, lossFunc, optimizer, epochs=25)

In [0]:
plotCost(history)

Select the best model (i.e. the weights file saved on the max epoch) to test.

In [0]:
saved_model = torch.load('src_model_1.pt')

In [0]:
computeTestSetAccuracy(saved_model, lossFunc)